In [1]:
import pandas as pd
import torch
from preprocessing import *
from RNN import RNN
import time
set_seed(42)

In [2]:
df = pd.read_csv(r'output_csv\HCMUT-SuperNodeXP-2017-1.0.swf.csv')
# df = df.head(100)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15886 entries, 0 to 15885
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   job_id                    15886 non-null  float64
 1   submit_time               15886 non-null  float64
 2   wait_time                 15886 non-null  float64
 3   run_time                  15886 non-null  float64
 4   num_allocated_processors  15886 non-null  float64
 5   avg_cpu_time_used         15886 non-null  float64
 6   used_memory               15886 non-null  float64
 7   requested_processors      15886 non-null  float64
 8   requested_time            15886 non-null  float64
 9   requested_memory          15886 non-null  float64
 10  status                    15886 non-null  float64
 11  user_id                   15886 non-null  float64
 12  group_id                  15886 non-null  float64
 13  executable_id             15886 non-null  float64
 14  queue_

In [3]:
feature_columns = ['requested_processors', 'requested_time', 'avg_cpu_time_used', 'used_memory', 'submit_time', 'wait_time', 'user_id', 'group_id', 'executable_id', 'queue_id']
target_column = 'run_time'

# Hyperparameters
input_dim = len(feature_columns)
num_hidden = 64
num_layers = 3
dropout = 0.2
num_epochs = 10
lr = 0.001
batch_size = 128  
seq_len = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataloaders, scaler = prepare_data_seq(df, feature_columns, target_column, seq_len=seq_len, batch_size=batch_size)

In [4]:
start = time.time()

# Initialize the model
model = RNN(input_dim, num_hidden, num_layers, dropout).to(device)

# Train the model
model.train_model(model, dataloaders['train'], dataloaders['val'], epochs=num_epochs, lr=lr, scaler=scaler)

end = time.time()

# Evaluate the model
rmse, mae, mse, r2 = model.evaluate_model(model, dataloaders['test'], scaler, num_features=input_dim)

print(f'\nFinal Test Metrics:')
print(f'Test RMSE: {rmse:.4f}, Test MAE: {mae:.4f}, Test MSE: {mse:.4f}, Test R2: {r2:.4f}')
print(f'Training Time: {end - start:.2f} seconds')

Epoch [1/10], Train Loss: 0.2670, Val Loss: 0.3577
Val RMSE: 145538.3905, Val MAE: 97915.9709, Val MSE: 21181423120.0757, Val R2: -0.4359
Model saved at epoch 1 with validation loss: 0.3577
Epoch [2/10], Train Loss: 0.2464, Val Loss: 0.2666
Val RMSE: 123373.7684, Val MAE: 82946.0406, Val MSE: 15221086719.7929, Val R2: -0.0319
Model saved at epoch 2 with validation loss: 0.2666
Epoch [3/10], Train Loss: 0.2224, Val Loss: 0.2669
Val RMSE: 120341.1458, Val MAE: 82478.5447, Val MSE: 14481991366.8126, Val R2: 0.0182
Epoch [4/10], Train Loss: 0.2143, Val Loss: 0.2463
Val RMSE: 114810.6718, Val MAE: 78743.6063, Val MSE: 13181490361.7762, Val R2: 0.1064
Model saved at epoch 4 with validation loss: 0.2463
Epoch [5/10], Train Loss: 0.2072, Val Loss: 0.2167
Val RMSE: 108084.4413, Val MAE: 73634.4595, Val MSE: 11682246450.6907, Val R2: 0.2080
Model saved at epoch 5 with validation loss: 0.2167
Epoch [6/10], Train Loss: 0.1978, Val Loss: 0.1938
Val RMSE: 102077.6243, Val MAE: 69316.0303, Val MSE: 1